In [1]:
%matplotlib inline

In [2]:
from keras.applications import VGG16
#from keras.applications import InceptionResNetV2

Using Theano backend.
Using cuDNN version 5103 on context None
Mapped name None to device cuda: Tesla K80 (30FC:00:00.0)


In [3]:
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications.vgg16 import preprocess_input
import numpy as np
from PIL import Image

In [4]:
base_model = VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(3, 224, 224), pooling='avg', classes=1000)

In [5]:
#train_path = "/home/arman/deep-learning/fastai/data/dogscats/train/"
#valid_path = "/home/arman/deep-learning/fastai/data/dogscats/valid/"
#test_path = "/home/arman/deep-learning/fastai/data/kaggle/test/"
#model_path = "/home/arman/deep-learning/fastai/data/dogscats/models/"

train_path = '/home/arman/deep-learning/kaggle-competitions/state-farm/data/train/'
valid_path = '/home/arman/deep-learning/kaggle-competitions/state-farm/data/valid/'
test_path = '/home/arman/deep-learning/kaggle-competitions/state-farm/data/test/'
model_path = '/home/arman/deep-learning/kaggle-competitions/state-farm'


In [6]:
x = base_model.output
# x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
for layer in base_model.layers:
    layer.trainable = False

In [8]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [9]:
train_gen = image.ImageDataGenerator()
valid_gen = image.ImageDataGenerator()
train_batches = train_gen.flow_from_directory(train_path, target_size=(224,224), shuffle=True, batch_size=8, class_mode='categorical')
valid_batches = valid_gen.flow_from_directory(valid_path, target_size=(224,224), shuffle=True, batch_size=8, class_mode='categorical')

Found 20324 images belonging to 10 classes.
Found 2100 images belonging to 10 classes.


In [10]:
model.fit_generator(train_batches, steps_per_epoch=int(np.ceil(train_batches.samples/32)), epochs=1, 
                    validation_data = valid_batches, validation_steps=int(np.ceil(valid_batches.samples/32)) )

Epoch 1/1
636/636 [==============================] - 122s - loss: 14.4363 - val_loss: 14.9276


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 224, 224)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 224, 224)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 224, 224)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 112, 112)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 112, 112)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 112, 112)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 56, 56)       0         
__________

In [12]:
model.save_weights(model_path + 'state_farm.h5')

In [50]:
test_gen = image.ImageDataGenerator()
test_batches = test_gen.flow_from_directory(test_path, target_size=(224,224), shuffle=False, batch_size=8, class_mode=None)

Found 12500 images belonging to 1 classes.


In [51]:
steps = int(np.ceil(test_batches.samples/8))

In [52]:
pred_gen = model.predict_generator(test_batches, steps)

In [53]:
pred_gen.shape

(12500, 2)

In [ ]:
#Image.open(test_path + test_batches.filenames[3])
#print(test_batches.filenames[0])

In [54]:
isdog = pred_gen[:, 1]
isdog = isdog.clip(min=0.05, max=0.95)

In [55]:
ids = [int(element[5:][0:-4]) for element in test_batches.filenames]
results = np.stack((ids, isdog), -1)

In [60]:
submission_file_name = '/home/arman/deep-learning/fastai/deeplearning_keras2/nbs/submission2.csv'

In [62]:
np.savetxt(submission_file_name, results, fmt='%d,%.5f', header='id,label', comments='')